# Getting Started

## Overview
This semester, all homeworks will be conducted through Google Colab notebooks. All code for the homework assignment will be written and run in this notebook. Running in Colab will automatically provide a GPU, but you may also run this notebook locally by following [these instructions](https://research.google.com/colaboratory/local-runtimes.html) if you wish to use your own GPU.

You will save images in the notebooks to use and fill out a given LaTeX template which will be submitted to Gradescope, along with your notebook code.

## Using Colab
On the left-hand side, you can click the different icons to see a Table of Contents of the assignment, as well as local files accessible through the notebook.

Make sure to go to **Runtime -> Change runtime type** and select **GPU** as the hardware accelerator. This allows you to use a GPU. Run the cells below to get started on the assignment. Note that a session is open for a maximum of 12 hours, and using too much GPU compute may result in restricted access for a short period of time. Please start the homework early so you have ample time to work.

**If you load this notebook by clicking "Open in Colab" from github, you will need to save it to your own Google Drive to keep your work.**

## General Tips
In each homework problem, you will implement autoregressive models and run it on various datasets. Oftentime you will run it on two datasets (dataset 1 and dataset 2). In these cases, the expected outputs for dataset 1 are already provided to help as a sanity check.

Feel free to print whatever output (e.g. debugging code, training code, etc) you want, as the graded submission will be the submitted pdf with images.

After you complete the assignment, download all of the images outputted in the results/ folder and upload them to the figure folder in the given latex template.

There is a lot of freedom in this homework to design write and design your own models. Hyperparameters are given as a guide to show what worked for us, but feel free to explore and use what you find is best!

Run the cells below to download and load up the starter code.

In [ ]:
!unzip -qq data/hw1_data.zip -d data/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data

import os.path as osp
from torch.utils.data import DataLoader
from typing import Tuple
from tqdm import tqdm
from time import time
from transformers import get_cosine_schedule_with_warmup

from deepul.hw1_helper import (
    # Q1
    visualize_q1_data,
    q1_sample_data_1,
    q1_sample_data_2,
    q1_save_results,
    # Q2
    q2a_save_results,
    q2b_save_results,
    visualize_q2a_data,
    visualize_q2b_data,
    # Q3
    q3ab_save_results,
    q3c_save_results,
    # Q4
    q4a_save_results,
    q4b_save_results,
    # Q5
    load_text_data,
    visualize_q5_data,
    q5a_save_results,
    # Q6
    load_colored_mnist_text,
    load_pretrain_vqvae,
    visualize_q6_data,
    q6a_save_results,
)
from deepul import pytorch_util
from deepul import utils

GPU_ID = 1

# Question 1: 1D Data

In this question, we will train simple generative models on discrete 1D data.

Execute the cell below to visualize our datasets

In [ ]:
visualize_q1_data(dset_type=1)
visualize_q1_data(dset_type=2)

## Part (a) Fitting a Histogram

Let $\theta = (\theta_0, \dots, \theta_{d-1}) \in \mathbb{R}^{d}$ and define the model $p_\theta(x) = \frac{e^{\theta_x}}{\sum_{x'}e^{\theta_{x'}}}$

Fit $p_\theta$ with maximum likelihood via stochastic gradient descent on the training set, using $\theta$ initialized to zero. Use your favorite version of stochastic gradient descent, and optimize your hyperparameters on a validation set of your choice.

**You will provide these deliverables**


1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. Plot the model probabilities in a bar graph with $\{0,\dots,d-1\}$ on the x-axis and a real number in $[0,1]$ on the y-axis.




Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [ ]:
train_data, test_data = q1_sample_data_1()
print(train_data[:10])
print(train_data.dtype)

In [ ]:
class Histogram(pytorch_util.MyModule):
    def __init__(self, d: int):
        super().__init__()
        self.theta = nn.Parameter(torch.zeros(d))
        self.print_n_params()

    @property
    def logits(self):
        return self.theta

    @property
    def device(self):
        return self.theta.device

    def forward(self, x: torch.Tensor):
        """
        x: (batch,)

        Returns
        - logits tensor of shape (batch, d)
        """
        return self.logits.unsqueeze(0).expand(x.shape[0], -1)

In [ ]:
def q1_a(train_data, test_data, d, dset_id):
  """
  train_data: An (n_train,) numpy array of integers in {0, ..., d-1}
  test_data: An (n_test,) numpy array of integers in {0, ..., d-1}
  d: The number of possible discrete values for random variable x
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
             used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (d,) of model probabilities
  """
  pytorch_util.set_gpu_mode("cuda", GPU_ID)
  device = pytorch_util.device
  pytorch_util.seed_rng(0)

  train_data = torch.from_numpy(train_data).to(device)
  test_data = torch.from_numpy(test_data).to(device)
  train_losses, test_losses = [], []

  model = Histogram(d)
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)

  def evaluate():
    with torch.no_grad():
      logits = model(test_data)
      loss = F.cross_entropy(logits, test_data)
    return loss.cpu().item()

  test_losses.append(evaluate())
  epochs = 100
  for _ in tqdm(range(epochs)):
    # train loss
    for train_mbatch in train_dataloader:
      optimizer.zero_grad()
      logits = model(train_mbatch)
      loss = F.cross_entropy(logits, train_mbatch)
      loss.backward()
      optimizer.step()
      train_losses.append(loss.cpu().item())

    # test loss
    test_losses.append(evaluate())

  with torch.no_grad():
    probs = F.softmax(model.logits, 0).cpu().numpy()

  return np.array(train_losses), np.array(test_losses), probs

### Results

Once you've implemented `q1_a`, execute the cells below to visualize and save your results



In [ ]:
q1_save_results(1, 'a', q1_a)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
q1_save_results(2, 'a', q1_a)
with torch.no_grad():
    torch.cuda.empty_cache()

## Part (b) Fitting Discretized Mixture of Logistics

Let us model $p_\theta(x)$ as a **discretized** mixture of 4 logistics such that $p_\theta(x) = \sum_{i=1}^4 \pi_i[\sigma((x+0.5 - \mu_i)/s_i) - \sigma((x-0.5-\mu_i)/s_i)]$

For the edge case of when $x = 0$, we replace $x-0.5$ by $-\infty$, and for $x = 99$, we replace $x+0.5$ by $\infty$.

You may find the [PixelCNN++](https://arxiv.org/abs/1701.05517) helpful for more information on discretized mixture of logistics.

**Provide the same set of corresponding deliverables as part (a)**

Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

We shall derive the log-probability $\log p_\theta(x)$. Let's define the following functions:
$$
\Delta \sigma_i(x) = \begin{cases}
\sigma((x+0.5-\mu_i)/s_i), & x = 0 \\
1-\sigma((x-0.5-\mu_i)/s_i), & x = 99 \\
\sigma((x+0.5-\mu_i)/s_i) - \sigma((x-0.5-\mu_i)/s_i)), & \text{otherwise}.
\end{cases}
$$
We then define a weight vector $w \in \mathbb{R}^n$ such that $\pi_i = \left. e^{w_i} \middle/ \sum_j e^{w_j} \right.$. Then, the log-probability is
\begin{align*}
\log p_\theta(x) &= \log \dfrac{\displaystyle \sum_{i=1}^n e^{w_i + \log \Delta \sigma_i(x)} } {\displaystyle \sum_{i=1}^n e^{w_i}} \\
&= \log \sum_{i=1}^n e^{w_i + \log \Delta \sigma_i(x)} - \log \sum_{i=1}^n e^{w_i} .
\end{align*}
We recognize this as the difference of two log-sum-exp functions.

We shall now show that $\log \Delta \sigma_i(x)$ has the following form:
$$
\log \Delta \sigma_i(x) = \begin{cases}
\log \sigma((x+0.5-\mu_i) / s_i), & x = 0 \\
\log \sigma (-(x-0.5-\mu_i) / s_i), & x = 99 \\
-(x+0.5-\mu_i)/s_i - (\log\sigma)^{-1}(-1/s_i) + \log\sigma((x+0.5-\mu_i)/s_i) + \log\sigma((x-0.5-\mu_i)/s_i), & \text{otherwise},
\end{cases}
$$
where $(\log\sigma)^{-1}(x) = -\log(e^{-x}-1)$ (*inverse log-sigmoid function*).

For compactness, we define $\Delta x_i = x - \mu_i$. For $x = 0$, we have:
$$
\log \Delta\sigma_i(x) = \log \sigma((\Delta x_i+0.5)/s_i).
$$

Similarly, for $x = 99$, we have:
$$
\log \Delta\sigma_i(x) = \log (1 - \sigma((\Delta x_i-0.5)/s_i)) = \log \sigma(-(\Delta x_i-0.5)/s_i)),
$$
where we take advantage of the fact that $1 - \sigma(x) = \sigma(-x)$.

Finally, for $0 < x < 99$, we have:
\begin{align*}
\log \Delta\sigma_i(x) &= \log\left( \frac{1}{1 + e^{-(\Delta x_i + 0.5)/s_i}} - \frac{1}{1 + e^{-(\Delta x_i - 0.5)/s_i}} \right) \\
&= \log \frac{ e^{-(\Delta x_i - 0.5)/s_i} - e^{-(\Delta x_i + 0.5)/s_i} }{ (1 + e^{-(\Delta x_i + 0.5)/s_i}) (1 + e^{-(\Delta x_i - 0.5)/s_i}) } \\
&= -\frac{\Delta x_i + 0.5}{s_i} + \log(e^{1/s_i} - 1) + \log\sigma((\Delta x_i+0.5)/s_i) + \log\sigma((\Delta x_i-0.5)/s_i) \\
&= -\frac{\Delta x_i + 0.5}{s_i} - (\log\sigma)^{-1}(-1/s_i) + \log\sigma((\Delta x_i+0.5)/s_i) + \log\sigma((\Delta x_i-0.5)/s_i) .
\end{align*}

In [ ]:
def logsigmoid_inverse(x: torch.Tensor):
    return -torch.log(torch.expm1(-x))


class DML(pytorch_util.MyModule):
    def __init__(self, n_mix: int, d: int):
        super().__init__()
        self.n_mix = n_mix
        self.d = d
        self.log_weights = nn.Parameter(torch.zeros(self.n_mix))
        self.locs = nn.Parameter(torch.arange(self.n_mix).float() / (self.n_mix - 1) * self.d)
        self.log_inv_scales = nn.Parameter(torch.randn(self.n_mix))
        self.print_n_params()

    def log_discretized_logistic(self, x: torch.Tensor):
        x_unsq = x.unsqueeze(1) # [d, 1]
        x_shifted = x_unsq.float() - self.locs # [d, n_mix]
        scales_inv = torch.exp(self.log_inv_scales)  # [n_mix]
        left = F.logsigmoid(scales_inv * (x_shifted+0.5))
        right = F.logsigmoid(-scales_inv * (x_shifted-0.5))
        middle = (-scales_inv*(x_shifted+0.5) - logsigmoid_inverse(-scales_inv)
                  + F.logsigmoid(scales_inv * (x_shifted+0.5)) + F.logsigmoid(scales_inv * (x_shifted-0.5)))
        return torch.where(x_unsq == 0, left, torch.where(x_unsq == self.d-1, right, middle))

    @property
    def logits(self) -> torch.Tensor:
        x = torch.arange(self.d, device=self.log_weights.device)  # [d]
        log_disc_logistic = self.log_discretized_logistic(x)  # [d, n_mix]
        logits_th = torch.logsumexp(self.log_weights + log_disc_logistic, 1) - torch.logsumexp(self.log_weights, 0)
        return logits_th

    @property
    def device(self):
        return self.log_weights.device

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (batch,)

        Returns
        - logits tensor of shape (batch, d)
        """
        return self.logits.unsqueeze(0).expand(x.shape[0], -1)

In [ ]:
def q1_b(train_data, test_data, d, dset_id):
  """
  train_data: An (n_train,) numpy array of integers in {0, ..., d-1}
  test_data: An (n_test,) numpy array of integers in {0, ..., d-1}
  d: The number of possible discrete values for random variable x
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (d,) of model probabilities
  """
  pytorch_util.set_gpu_mode("cuda", GPU_ID)
  device = pytorch_util.device
  pytorch_util.seed_rng(0)
    
  train_data = torch.from_numpy(train_data).to(device)
  test_data = torch.from_numpy(test_data).to(device)
  train_losses, test_losses = [], []

  n_mix = 4
  model = DML(n_mix, d)
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=3e-2)
  train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)

  def evaluate():
    with torch.no_grad():
      logits = model(test_data)
      loss = F.cross_entropy(logits, test_data)
      return loss.cpu().item()

  test_losses.append(evaluate())
  epochs = 30 if dset_id == 1 else 10
  for _ in tqdm(range(epochs)):
    # train loss
    for train_mbatch in train_dataloader:
      optimizer.zero_grad()
      logits = model(train_mbatch)
      loss = F.cross_entropy(logits, train_mbatch)
      loss.backward()
      optimizer.step()
      train_losses.append(loss.cpu().item())
    
    # test loss
    test_losses.append(evaluate())

  with torch.no_grad():
    probs = F.softmax(model.logits, 0).cpu().numpy()
  return train_losses, test_losses, probs

### Results

Once you've implemented `q1_b`, execute the cells below to visualize and save your results



In [ ]:
q1_save_results(1, 'b', q1_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
q1_save_results(2, 'b', q1_b)
with torch.no_grad():
    torch.cuda.empty_cache()

# Question 2 PixelCNNs

Now, you will train more powerful PixelCNN models on the shapes dataset and MNIST. In addition, we will extend to modeling colored datasets.

Run the cell below to visualize the two datasets binary datasets

In [ ]:
visualize_q2a_data(1)
visualize_q2a_data(2)

In [ ]:
train_data, test_data = utils.load_pickled_data("data/shapes.pkl")
print(train_data.dtype)
print(train_data.shape)
print(test_data.shape)
print(train_data[0].squeeze())

## Part (a) PixelCNN on Shapes and MNIST
In this part, implement a simple PixelCNN architecture to model binary MNIST and shapes images (same as Q2(b), but with a PixelCNN).

We recommend the following network design:
* A $7 \times 7$ masked type A convolution
* $5$ $7 \times 7$ masked type B convolutions
* $2$ $1 \times 1$ masked type B convolutions
* Appropriate ReLU nonlinearities in-between
* 64 convolutional filters

And the following hyperparameters:
* Batch size 128
* Learning rate $10^{-3}$
* 10 epochs
* Adam Optimizer (this applies to all PixelCNN models trained in future parts)

Your model should output logits, after which you could apply a sigmoid over 1 logit, or a softmax over two logits (either is fine). It may also help to scale your input to $[-1, 1]$ before running it through the network. 

Training on the shapes dataset should be quick, and MNIST should take around 10 minutes

Checkout the Paper for more details: https://arxiv.org/abs/1601.06759

**You will provide these deliverables**


1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. 100 samples from the final trained model



Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [ ]:
class MaskedConv2d(nn.Conv2d):
    def __init__(self, mask_type: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        assert mask_type in {"A", "B"}
        self.register_buffer("mask", self.weight.data.clone())
        _, _, h, w = self.weight.size()
        self.mask.fill_(0.)
        self.mask[:, :, h // 2, :w // 2 + (mask_type == "B")] = 1.
        self.mask[:, :, :h // 2, :] = 1.
        self._mask_weight()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        self._mask_weight()
        return super().forward(x)

    def _mask_weight(self):
        self.weight.data.mul_(self.mask)

Let's verify the masked convolution implementation is correct.

In [ ]:
conv_a = MaskedConv2d("A", 1, 1, 7, padding=(7-1) // 2)
conv_b = MaskedConv2d("B", 1, 1, 7, padding=(7-1) // 2)

print("Mask A")
print(conv_a.mask.squeeze())
print(conv_a.weight.data.squeeze())

print()
print("===========")
print()

print("Mask B")
print(conv_b.mask.squeeze())
print(conv_b.weight.data.squeeze())

In [ ]:
class BinaryPixelCNN(pytorch_util.MyModule):    
    def __init__(self, n_filters: int):
        super().__init__()        
        self.convs = nn.ModuleList(
              [MaskedConv2d("A", 1,         n_filters, 7, padding=(7-1) // 2)]
            + [MaskedConv2d("B", n_filters, n_filters, 7, padding=(7-1) // 2) for _ in range(5)]
            + [MaskedConv2d("B", n_filters, n_filters, 1)]
        )
        self.conv_out = MaskedConv2d("B", n_filters, 1, 1)
        self.print_n_params()

    @property
    def device(self):
        return self.convs[0].weight.device # self.conv_a.weight.device

    def forward(self, x):
        x = 2*x.permute(0, 3, 1, 2).float() - 1
        for conv in self.convs:
            x = F.relu(conv(x))
        return self.conv_out(x).permute(0, 2, 3, 1)
        

    def sample(self, size, n_samples):
        with torch.no_grad():
            samples = torch.zeros(n_samples, *size, 1, device=self.device, dtype=torch.uint8)
            for i in range(size[0]):
                for j in range(size[1]):
                    logits = self(samples)[:, i, j, 0]
                    sampled_pixels = torch.bernoulli(torch.sigmoid(logits)).type(torch.uint8)
                    samples[:, i, j, 0] = sampled_pixels
        return samples.cpu().numpy()

In [ ]:
def q2_a(train_data, test_data, image_shape, dset_id):
  """
  train_data: A (n_train, H, W, 1) uint8 numpy array of binary images with values in {0, 1}
  test_data: A (n_test, H, W, 1) uint8 numpy array of binary images with values in {0, 1}
  image_shape: (H, W), height and width of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (100, H, W, 1) of samples with values in {0, 1}
  """
  pytorch_util.set_gpu_mode("cuda", GPU_ID)
  device = pytorch_util.device
  pytorch_util.seed_rng(0)
    
  train_data = torch.from_numpy(train_data).to(device)
  test_data = torch.from_numpy(test_data).to(device)
  train_losses, test_losses = [], []

  model = BinaryPixelCNN(64)
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)

  def evaluate():
    with torch.no_grad():
      logits = model(test_data)
      loss = F.binary_cross_entropy_with_logits(logits, test_data.float())
      return loss.cpu().item()

  test_losses.append(evaluate())
  epochs = 15 if dset_id == 1 else 5
  for _ in tqdm(range(epochs)):
    # train loss
    for train_mbatch in train_dataloader:
      train_mbatch = train_mbatch.to(device)
      optimizer.zero_grad()
      logits = model(train_mbatch)
      loss = F.binary_cross_entropy_with_logits(logits, train_mbatch.float())
      loss.backward()
      optimizer.step()
      train_losses.append(loss.cpu().item())

    # test loss
    test_losses.append(evaluate())

  return train_losses, test_losses, model.sample(image_shape, 100)

### Results

Once you've implemented `q2_a`, execute the cells below to visualize and save your results



In [ ]:
q2a_save_results(1, q2_a)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
q2a_save_results(2, q2_a)
with torch.no_grad():
    torch.cuda.empty_cache()

## Part (b) PixelCNN on Colored Shapes and MNIST: Independent Color Channels

For the next part, we'll work with color images (shapes and MNIST). Run the cell below to visualize the dataset.

In [ ]:
visualize_q2b_data(1)
visualize_q2b_data(2)

In [ ]:
train_data, test_data = utils.load_pickled_data("data/shapes_colored.pkl")
print(train_data.dtype)
print(train_data.shape)
print(test_data.shape)
train_data_one_hot = F.one_hot(torch.from_numpy(train_data).long(), num_classes=4)
print(train_data_one_hot.shape)

print(train_data[0, :2, :3].squeeze())
print("=======")
print(train_data_one_hot[0, :2, :3])

Now, implement a PixelCNN to support RGB color channels (or augment your existing implementation). **First, implement a PixelCNN that assumes color channels as independent.** More formally, we model the following parameterized distribution:

$$p_\theta(x) = \prod_{i=1}^{HW}\prod_{c=1}^C p_\theta(x_i^c | x_{<i})$$

Here are some tips that you may find useful for designing and training these models:
* You will need a 4-way softmax for every prediction, as opposed to a 256-way softmax in the PixelCNN paper, since the dataset is quantized to two bits per color channel
* You can set the number of filters for each convolutions to 120. You can use the ReLU nonlinearity throughout.
* Use a stack of 8 residual block architecture from [Figure 5](https://arxiv.org/abs/1601.06759) but with 7 x 7 masked convolutions in the middle instead of 3 x 3 masked convolutions
* Consider using [layer normalization](https://arxiv.org/abs/1607.06450) to improve performance. However, be careful to maintain the autoregressive property.
* With a learning rate of $10^{-3}$ and a batch size of 128, it should take a few minutes to run on the shapes dataset, and about 50-60 minutes on MNIST.

**You will provide these deliverables**


1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. 100 samples from the final trained model



Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [ ]:
class BaseColorPixelCNN(pytorch_util.MyModule):
    def __init__(self):
        super().__init__()
        self.n_colors = 4
        self.n_channels = 3

    def sample(self, size: Tuple[int, int, int], n_samples: int) -> np.ndarray:
        """
        size: tuple corresponding to (H, W, C)
        n_samples: number of samples

        Returns
        - a numpy array of size (n_samples, H, W, C) of samples with values in {0, 1, 2, 3}
        """
        with torch.no_grad():
            samples = torch.zeros(n_samples, *size, device=self.device, dtype=torch.uint8)
            for i in range(size[0]):
                for j in range(size[1]):
                    logits = self(samples)[:, i, j] # (n_samples, C, colors)
                    probs = F.softmax(logits, -1) # (n_samples, C, colors)
                    probs_flat = probs.view(-1, self.n_colors) # (n_samples * C, colors)
                    sampled_pixels_flat = torch.multinomial(probs_flat, num_samples=1) # (n_samples * C, 1)
                    sampled_pixels = sampled_pixels_flat.view(logits.shape[:2]) # (n_samples, C)
                    samples[:, i, j] = sampled_pixels
        return samples.cpu().numpy()

First, we implement a residual block, which alternates between layer normalization (optional), ReLU non-linearity, and masked type B convolution.

To maintain the autoregressive property, we compute the layer normalization over the *channels* of the feature $x_{b,i,j,c}$, i.e., the mean and variance are computed as:
\begin{align*}
\mu_{b,i,j} &= \mathbb{E}_{c, (i', j') \in W(i, j)} [x_{b,i',j',c}] \\
\sigma_{b,i,j}^2 &= \mathrm{var}_{c, (i',j') \in W(i,j)} (x_{b,i',j',c})
\end{align*}

In [ ]:
class AutoregressiveLayerNorm(nn.Module):
    def __init__(self, channels: int, kernel_size: int = 7):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(channels))
        self.bias = nn.Parameter(torch.zeros(channels))
        self.register_buffer("mean_filter", torch.zeros(1, channels, kernel_size, kernel_size))
        self.mean_filter[:, :, kernel_size // 2, :kernel_size // 2 + 1] = 1.
        self.mean_filter[:, :, :kernel_size // 2, :] = 1.
        self.mean_filter /= self.mean_filter.sum()
        self.kernel_size = kernel_size

    def forward(self, x: torch.Tensor):
        """
        x: a (batch, channels, H, W) float tensor

        Returns
        - a (batch, channels, H, W) float tensor
        """
        means = F.conv2d(x, self.mean_filter, padding=(self.kernel_size-1) // 2)  # (batch, 1, h, w)

        deltas = x - means # (batch, channels, H, W)
        sq_deltas = torch.square(x - means) # (batch, channels, H,  W)
        vars = F.conv2d(sq_deltas, self.mean_filter, padding=(self.kernel_size-1) // 2)  # (batch, 1, H, W)

        x_whitened = deltas / torch.sqrt(vars + 1e-5)  # (batch, channels, H, W)

        return self.weight[:, None, None] * x_whitened + self.bias[:, None, None]

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, n_filters: int, layer_norm: bool = True):
        super().__init__()

        self.convs = nn.ModuleList([
            MaskedConv2d("B", n_filters, n_filters, 1),
            MaskedConv2d("B", n_filters, n_filters, 7, padding=(7-1) // 2),
            MaskedConv2d("B", n_filters, n_filters, 1)
        ])

        #layer_norm_ctor = nn.LayerNorm if layer_norm else nn.Identity
        layer_norm_ctor = AutoregressiveLayerNorm if layer_norm else nn.Identity
        self.lns = nn.ModuleList([layer_norm_ctor(n_filters) for _ in range(3)])

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: a (batch, n_filters, H, W) float tensor

        Returns
        - a (batch, n_filters, H, W) float tensor
        """
        y = x

        for conv, ln in zip(self.convs, self.lns):
            #y = ln(y.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
            y = ln(y)
            y = F.relu(y)
            y = conv(y)

        return x + y

We first implement a `FullySeparatedColorPixelCNN` which applies which applies 3 `ChannelPixelCNN`'s to the image, one per channel.

In [ ]:
class ChannelPixelCNN(nn.Module):
    def __init__(self, n_filters: int, layer_norm: bool = True):
        super().__init__()
        self.n_channels = 3
        self.n_colors = 4
        self.conv_a = MaskedConv2d("A", self.n_channels, n_filters, 7, padding=(7-1) // 2)
        self.resids = nn.ModuleList([ResidualBlock(n_filters, layer_norm) for _ in range(8)])
        self.conv_proj = MaskedConv2d("B", n_filters, self.n_colors, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: a (batch, H, W, C) uint8 tensor of color images with values in {0, 1, 2, 3}

        Returns
        - a tensor of logits with shape (batch, H, W, 4)
        """
        x = 2/3*x.permute(0, 3, 1, 2).float() - 1 # (batch, C, H, W)
        x = self.conv_a(x)
        for resid in self.resids:
            x = resid(x)
        return self.conv_proj(x).permute(0, 2, 3, 1)


class FullySeparatedColorPixelCNN(BaseColorPixelCNN):
    def __init__(self, n_filters: int, layer_norm: bool = True):
        super().__init__()
        self.cnns = nn.ModuleList([ChannelPixelCNN(n_filters, layer_norm) for _ in range(self.n_channels)])
        self.print_n_params()

    @property
    def device(self):
        return self.cnns[0].conv_a.weight.device

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: A (batch, H, W, C) uint8 tensor of color images with values in {0, 1, 2, 3}

        Returns
        - a tensor of logits with shape (batch, H, W, C, 4)
        """
        logits = torch.empty(*x.shape, self.n_colors, device=x.device) # (batch, H, W, C, colors)
        for i, cnn in enumerate(self.cnns):
            logits[:, :, :, i] = cnn(x)
        return logits

We also implement a `MultiTailedColorPixelCNN`, which applies 3 separate modules consisting of masked type A convolution and two residual blocks. The outputted features are then concatenated along the batch dimension and then processed through six residual blocks to form the logits.

In [ ]:
class MultiTailedColorPixelCNN(BaseColorPixelCNN):
    def __init__(self, n_filters: int, layer_norm: bool = True):
        super().__init__()
        self.convs_a = nn.ModuleList([MaskedConv2d("A", self.n_channels, n_filters, 7, padding=(7-1) // 2) for _ in range(self.n_channels)])
        self.resids_separate = nn.ModuleList([
            nn.ModuleList([ResidualBlock(n_filters, layer_norm) for _ in range(2)])
        for _ in range(self.n_channels)])
        self.resids_shared = nn.ModuleList([ResidualBlock(n_filters, layer_norm) for _ in range(6)])
        self.conv_out = MaskedConv2d("B", n_filters, self.n_colors, 1)
        self.print_n_params()

    @property
    def device(self):
        return self.conv_out.weight.device
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: a (batch, H, W, C) uint8 tensor of color images with values in {0, 1, 2, 3}

        Returns
        - a tensor of logits with shape (batch, H, W, C, 4)
        """
        x = 2/3*x.permute(0, 3, 1, 2).float() - 1. # (batch, C, H, W)

        xs = []
        for conv_a, resids in zip(self.convs_a, self.resids_separate):
            y = x
            y = conv_a(y)
            for resid in resids:
                y = resid(y)
            xs.append(y)

        x = torch.cat(xs, dim=0) # (C * batch, n_filters, H, W)
        for resid in self.resids_shared:
            x = resid(x)
        x = self.conv_out(x) # (C * batch, colors, H, W)
        x = x.view(self.n_channels, -1, self.n_colors, x.shape[-2], x.shape[-1]) # (C, batch, colors, H, W)
        x = x.permute(1, 3, 4, 0, 2) # (batch, H, W, C, 4)

        return x

Finally, we implement a `MultiHeadedColorPixelCNN`, which first applies six layers of residual blocks to the image. Then, the outputted features are fed through 3 separate modules (one per channel) of two residual blocks to form the logits.

In [ ]:
class MultiHeadedColorPixelCNN(BaseColorPixelCNN):
    def __init__(self, n_filters: int, layer_norm: bool = True):
        super().__init__()
        self.conv_a = MaskedConv2d("A", self.n_channels, n_filters, 7, padding=(7-1) // 2)
        self.resids_shared = nn.ModuleList([ResidualBlock(n_filters, layer_norm) for _ in range(6)])
        self.resids_separate = nn.ModuleList([
            nn.ModuleList([ResidualBlock(n_filters, layer_norm) for _ in range(2)])
            for _ in range(self.n_channels)])
        self.convs_out = nn.ModuleList([MaskedConv2d("B", n_filters, self.n_colors, 1) for _ in range(self.n_channels)])
        self.print_n_params()

    @property
    def device(self):
        return self.conv_a.weight.device
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: a (batch, H, W, C) uint8 tensor of color images with values in {0, 1, 2, 3}

        Returns
        - a tensor of logits with shape (batch, H, W, C, 4)
        """
        x = 2/3*x.permute(0, 3, 1, 2).float() - 1 # (batch, C, H, W)
        
        x = self.conv_a(x)
        for resid in self.resids_shared:
            x = resid(x)

        xs = []
        for resids, conv in zip(self.resids_separate, self.convs_out):
            y = x
            for resid in resids:
                y = resid(y)
            y = conv(y)
            xs.append(y)

        x = torch.stack(xs, dim=-1) # (batch, colors, H, W, C)
        x = x.permute(0, 2, 3, 4, 1) # (batch, H, W, C, colors)

        return x

In [ ]:
make_pixel_cnn = lambda: None # placeholder

def q2_b(train_data, test_data, image_shape, dset_id):
  """
  train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  image_shape: (H, W, C), height, width, and # of channels of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (100, H, W, C) of samples with values in {0, 1, 2, 3}
  """
  pytorch_util.set_gpu_mode("cuda", GPU_ID)
  device = pytorch_util.device
  pytorch_util.seed_rng(0)
    
  train_data = torch.from_numpy(train_data)
  test_data = torch.from_numpy(test_data)
  train_losses, test_losses = [], []

  model = make_pixel_cnn()
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
  test_dataloader = DataLoader(test_data, batch_size=128, shuffle=False)

  def evaluate():
    losses = []
    with torch.no_grad():
      for test_mbatch in test_dataloader:
        test_mbatch = test_mbatch.to(device)
        logits = model(test_mbatch)
        loss = F.cross_entropy(logits.reshape(-1, logits.size(-1)), test_mbatch.ravel())
        losses.append(loss.cpu().item())
    return np.mean(losses)

  test_losses.append(evaluate())
  epochs = 8 if dset_id == 1 else 3
  for _ in tqdm(range(epochs)):
    # train loss
    for train_mbatch in train_dataloader:
      train_mbatch = train_mbatch.to(device)
      optimizer.zero_grad()
      logits = model(train_mbatch)
      loss = F.cross_entropy(logits.reshape(-1, logits.size(-1)), train_mbatch.ravel())
      loss.backward()
      optimizer.step()

      train_losses.append(loss.cpu().item())

    # test loss
    test_losses.append(evaluate())

  return train_losses, test_losses, model.sample(image_shape, 100)

### Results

Once you've implemented `q2_b`, execute the cells below to visualize and save your results



#### Fully separated PixelCNN's

First, we try without layer norm.

In [ ]:
make_pixel_cnn = lambda: FullySeparatedColorPixelCNN(90, False)

In [ ]:
print("Fully separated, no layer norm")
q2b_save_results(1, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Fully separated, no layer norm")
q2b_save_results(2, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

Now, we try with layer norm.

In [ ]:
make_pixel_cnn = lambda: FullySeparatedColorPixelCNN(90, True)

In [ ]:
print("Fully separated, layer norm")
q2b_save_results(1, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Fully separated, layer norm")
q2b_save_results(2, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

#### Multi-tailed PixelCNN's

First, we try without layer norm.

In [ ]:
make_pixel_cnn = lambda: MultiTailedColorPixelCNN(128, False)

In [ ]:
print("Multi-tailed, no layer norm")
q2b_save_results(1, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Multi-tailed, no layer norm")
q2b_save_results(2, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

Now, we try with layer norm.

In [ ]:
make_pixel_cnn = lambda: MultiTailedColorPixelCNN(128, True)

In [ ]:
print("Multi-tailed, layer norm")
q2b_save_results(1, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Multi-tailed, layer norm")
#q2b_save_results(2, 'b', q2_b)
#with torch.no_grad():
#    torch.cuda.empty_cache()
print("Fails to run due to insufficient VRAM")

#### Multi-headed PixelCNN's

First, we try without layer norm.

In [ ]:
make_pixel_cnn = lambda: MultiHeadedColorPixelCNN(128, False)

In [ ]:
print("Multi-headed, no layer norm")
q2b_save_results(1, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Multi-headed, no layer norm")
q2b_save_results(2, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

Now, we try with layer norm.

In [ ]:
make_pixel_cnn = lambda: MultiHeadedColorPixelCNN(128, True)

In [ ]:
print("Multi-headed, layer norm")
q2b_save_results(1, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Multi-headed, layer norm")
q2b_save_results(2, 'b', q2_b)
with torch.no_grad():
    torch.cuda.empty_cache()

# Question 3: Causal Transformer - iGPT

Now we will move onto the current most popular and widespread autoregressive model, the transformer.

## Part (a) Autoregressive Transformer on Shapes and MNIST
In this part, implement a simple Autoregressive Transformer to model binary MNIST and shapes images (same as Q2(a), but with a Transformer). 

Some additional notes about your transformer implementation:
 * iGPT uses learned positional encodings. We recommend to use those here as well. However, you may also use sinusoidal positional encodings if you wish (see the [Attention is All You Need](https://arxiv.org/abs/1706.03762) paper)
 * Autoregressive transformer always predicts the **next** token, give prior tokens. iGPT has a special **\<bos\>** or beginning of sequence token at the start of every sequence every image. Make sure to include this in your implementation as well. You can generate unconditional sample by conditioning with the **\<bos\>** token.
 * While dropout is a common feature in transformer models, you do not need to add it (but may if you wish!).
 * Prebuilt transformers exist in some frameworks (i.e. PyTorch). Don't just use an off the shelf implementation as the point of the exercise is to better understand the transformer architecture. Building the transformer from the ground up (use primitives such as Linear/Dense layers, LayerNorm, GeLU, Embedding)
 * Learning rate warmup and cosine learning rate decay are often used when training transformers to improve training stability and improve performance. See if this helps your model! Try 1000 steps of warmup with a cosine learning rate decay.

Paper references
* [Attention Is All You Need](https://arxiv.org/abs/1706.03762) 
* [Generative Pretraining from Pixels](https://cdn.openai.com/papers/Generative_Pretraining_from_Pixels_V2.pdf) 
* [Language Models are Unsupervised Multitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)

We recommend the following network design parameters:
* $d_{model}$: 128
* heads: 4
* layers: 2
* GeLU nonlinearities

And the following hyperparameters:
* Batch size: 64 or 32 or 16 (whichever fits in your GPU)
* Learning rate: $10^{-3}$
* 15 epochs or more
* Adam Optimizer (this applies to all Transformers models trained in future parts)

**You will provide these deliverables**

1. Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2. Report the final test set performance of your final model
3. 100 samples from the final trained model



Let's build the causal self-attention layer bit by bit, first. We start by defining some hyperparameters and an arbitrary input $X \in \mathbb{R}^{L \times dh}$, where $L$ is the sequence length, $d$ is the embedding dimension, and $h$ is the number of attention heads.

In [ ]:
batch = 2
seq_length = 3
num_heads = 4
embed_dim = 5
x = torch.randn(batch, seq_length, embed_dim * num_heads)
print("X shape:", x.shape)
print("X")
print(x.data)

We then build a layer that maps $X$ to $Q$, $K$, and $V$, each of which is an $L \times d$ matrix. Namely, we rely on the linear property to efficiently compute them:
$$
\begin{bmatrix} Q \\ K \\ V \end{bmatrix} = \begin{bmatrix} W_Q \\ W_K \\ W_V \end{bmatrix} X,
$$
where each weight matrix is $L \times L$.

In [ ]:
qkv_layer = nn.Linear(embed_dim*num_heads, 3*embed_dim*num_heads, bias=False)
qkv = qkv_layer(x)
print("QKV shape:", list(qkv.size()))  # (batch, seq_length, 3*embed_dim*num_heads)

q, k, v = torch.split(qkv, embed_dim*num_heads, dim=2)  # (batch, seq_length, embed_dim*num_heads)
print("V shape:", list(v.size()))
print("V:")
print(v.data)

Since we're doing multi-head attention, we evenly partition $Q$, $K$, and $V$ between the different attention heads. The forthcoming math parallelizes over the heads.

In [ ]:
# q, k, v each of shape (batch, num_heads, seq_length, embed_dim)
q = q.view(batch, seq_length, num_heads, embed_dim).permute(0, 2, 1, 3)
k = k.view(batch, seq_length, num_heads, embed_dim).permute(0, 2, 1, 3)
v = v.view(batch, seq_length, num_heads, embed_dim).permute(0, 2, 1, 3)

print("V shape:", list(v.size()))
print("V:")
print(v.data)

Now, we compute the dot product score:
$$
\frac{QK^T}{\sqrt d},
$$
which is an $L \times L$ matrix.

In particular, note that the matrix has the form:
\begin{align*}
QK^T
&= \begin{bmatrix} q_1^T \\ \vdots \\ q_L^T \end{bmatrix} \begin{bmatrix} k_1 & \cdots & k_L \end{bmatrix} \\
&= \begin{bmatrix}
q_1^T k_1 & \cdots & q_1^T k_L \\
\vdots & \ddots & \vdots \\
q_L^T k_1 & \cdots & q_L^T k_L
\end{bmatrix} \\
&= \begin{bmatrix} q_1^T K^T \\ \vdots \\ q_L^T K^T \end{bmatrix} .
\end{align*}
Essentially, each row $i$ of the score matrix corresponds to some query $q_i$.

In [ ]:
score = torch.matmul(q, k.transpose(2, 3)) / np.sqrt(embed_dim)
print("Score shape:", list(score.size()))  # (batch, num_heads, seq_length, seq_length)
print("Score:")
print(score.data)

To make the attention layer causal, we mask the score tensor so that the $(i, j)$-entry is $-\infty$ if $i < j$. This is accomplished using the lower-triangular matrix:
$$
\begin{bmatrix}
0 & 1 & 1 & 1 & \cdots & 1 & 1 & 1 \\
0 & 0 & 1 & 1 & \cdots & 1 & 1 & 1 \\
0 & 0 & 0 & 1 & \cdots & 1 & 1 & 1 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & 0 & 1 & 1 \\
0 & 0 & 0 & 0 & \cdots & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & \cdots & 0 & 0 & 0
\end{bmatrix}
$$

In [ ]:
mask = ~(torch.tril(torch.ones(seq_length, seq_length)).bool())
print("Mask shape:", list(mask.size()))
print("Mask:")
print(mask.int())

We now mask $QK^T / \sqrt{d}$ into $S$.

In [ ]:
score.masked_fill_(mask, -torch.inf)
print("Masked score:")
print(score.data)

We then compute the weight matrix $W$ through $W = \mathrm{softmax}(S)$, where the softmax is done over the rows.

In [ ]:
weight = torch.softmax(score, dim=3)
print("Weight shape:", list(weight.size()))
print("Weight:")
print(weight.data)

Finally, we compute the output $Y \in \mathbb{R}^{L \times d}$ of the attention layer:
$$
Y = WV .
$$
Note that row $i$ of $Y$ corresponds to:
$$
\sum_{j=1}^L w_{i,j} v_j^T = w_i^T V
$$

In [ ]:
pred = weight @ v
print("Prediction shape:", list(pred.size()))
print("Prediction:")
print(pred.data)

Finally, we concatenate along the attention-head dimension.

In [ ]:
pred = pred.permute(0, 2, 1, 3)  # (batch, seq_length, num_heads, embed_dim)
pred = pred.reshape_as(x)
print("Prediction shape:", list(pred.size()))
print("Prediction:")
print(pred.data)

We now define the causal self-attention layer accordingly.

In [ ]:
class CausalSelfAttention(nn.Module):
    def __init__(self, seq_length: int, embed_dim: int, num_heads: int, dropout_prob: float):
        super().__init__()
        self.seq_length = seq_length
        self.embed_dim = embed_dim
        self.sqrt_embed_dim = np.sqrt(embed_dim)
        self.num_heads = num_heads
        self.dropout_prob = dropout_prob
        self.qkv_layer = nn.Linear(embed_dim*num_heads, 3*embed_dim*num_heads, bias=False)
        self.attn_dropout = nn.Dropout(dropout_prob)
        self.resid_dropout = nn.Dropout(dropout_prob)
        self.register_buffer("mask", ~(torch.tril(torch.ones(seq_length, seq_length)).bool()))

    def forward(self, x: torch.Tensor):
        """
        x: a (batch, seq_length, embed_dim*num_heads) float tensor

        Returns
        - a (batch, seq_length, embed_dim*num_heads) float tensor
        """
        qkv = self.qkv_layer(x)                                           # (batch, seq_length, 3*embed_dim*num_heads)
        q, k, v = torch.split(qkv, self.embed_dim*self.num_heads, dim=2)  # each tensor of shape (batch, seq_length, embed_dim*num_heads)

        batch = x.size(0)
        # q, k, v each of shape (batch, num_heads, seq_length, embed_dim)
        q = q.view(batch, self.seq_length, self.num_heads, self.embed_dim).permute(0, 2, 1, 3)
        k = k.view(batch, self.seq_length, self.num_heads, self.embed_dim).permute(0, 2, 1, 3)
        v = v.view(batch, self.seq_length, self.num_heads, self.embed_dim).permute(0, 2, 1, 3)

        score = q @ k.transpose(2, 3) / self.sqrt_embed_dim  # (batch, num_heads, seq_length, seq_length)

        # causal masking
        score.masked_fill_(self.mask, -torch.inf)

        weight = torch.softmax(score, dim=3)  # (batch, num_heads, seq_length, seq_length)
        weight = self.attn_dropout(weight)
        pred = weight @ v                     # (batch, num_heads, seq_length, embed_dim)
        pred = pred.permute(0, 2, 1, 3)       # (batch, seq_length, num_heads, embed_dim)
        pred = pred.reshape_as(x)             # (batch, seq_length, embed_dim*num_heads)
        pred = self.resid_dropout(pred)

        return pred

    def forward_with_cache(self, x: torch.Tensor, idx: int):
        """
        x: a (batch, embed_dim*num_heads) float tensor

        Returns
        - a (batch, embed_dim*num_heads) float tensor
        """
        batch = x.size(0)
        if idx == 0:
            self._k_cache = torch.zeros(batch, self.num_heads, self.seq_length, self.embed_dim, device=x.device)
            self._v_cache = torch.zeros_like(self._k_cache)

        qkv = self.qkv_layer(x)                                           # (batch, 3*embed_dim*num_heads)
        q, k, v = torch.split(qkv, self.embed_dim*self.num_heads, dim=1)  # each tensor of shape (batch, embed_dim*num_heads)

        # q, k, v each of shape (batch, num_heads, embed_dim)
        q = q.view(batch, self.num_heads, self.embed_dim)
        k = k.view(batch, self.num_heads, self.embed_dim)
        v = v.view(batch, self.num_heads, self.embed_dim)

        self._k_cache[:, :, idx] = k
        self._v_cache[:, :, idx] = v

        score = (q.unsqueeze(2) @ self._k_cache.transpose(2, 3)).squeeze(2) / self.sqrt_embed_dim # (batch, num_heads, seq_length)

        # causal masking
        score[:, :, idx+1:] = -float("inf")

        weight = torch.softmax(score, dim=2)                       # (batch, num_heads, seq_length)
        weight = self.attn_dropout(weight)
        pred   = (weight.unsqueeze(2) @ self._v_cache).squeeze(2)  # (batch, num_heads, embed_dim)
        pred   = pred.view_as(x)                                   # (batch, num_heads*embed_dim)
        pred   = self.resid_dropout(pred)

        return pred

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, seq_length: int, embed_dim: int, num_heads: int, dropout_prob: float):
        super().__init__()
        self.ln_in = nn.LayerNorm(embed_dim*num_heads)
        self.ln_out = nn.LayerNorm(embed_dim*num_heads)
        self.attn = CausalSelfAttention(seq_length, embed_dim, num_heads, dropout_prob)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim*num_heads, embed_dim*num_heads),
            nn.GELU(),
            nn.Linear(embed_dim*num_heads, embed_dim*num_heads),
            nn.Dropout(dropout_prob)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: a (batch, seq_length, embed_dim*num_heads) float tensor

        Returns
        - a (batch, seq_length, embed_dim*num_heads) float tensor
        """
        x = x + self.attn(self.ln_in(x))
        x = x + self.mlp(self.ln_out(x))
        return x

    def forward_with_cache(self, x: torch.Tensor, idx: int) -> torch.Tensor:
        """
        x: a (batch, embed_dim*num_heads) float tensor

        Returns
        - a (batch, embed_dim*num_heads) float tensor
        """
        x = x + self.attn.forward_with_cache(self.ln_in(x), idx)
        x = x + self.mlp(self.ln_out(x))
        return x

In [ ]:
class GPT(pytorch_util.MyModule):
    def __init__(
        self,
        num_tokens: int,
        num_pred_tokens: int,
        seq_length: int,
        embed_dim: int,
        num_heads: int,
        num_layers: int,
        dropout_prob: float = 0.,
        concat_embed: bool = False
    ):
        super().__init__()
        self.num_tokens = num_tokens
        self.num_pred_tokens = num_pred_tokens
        self.seq_length = seq_length
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.dropout_prob = dropout_prob
        self.concat_embed = concat_embed
        self.blocks = nn.ModuleList([TransformerBlock(seq_length, embed_dim, num_heads, dropout_prob) for _ in range(num_layers)])
        if concat_embed:
            self.token_embed_table = nn.Embedding(num_tokens, embed_dim*num_heads // 2)
            self.pos_embed_table = nn.Embedding(seq_length, embed_dim*num_heads // 2)
        else:
            self.token_embed_table = nn.Embedding(num_tokens, embed_dim*num_heads)
            self.pos_embed_table = nn.Embedding(seq_length, embed_dim*num_heads)
        self.embed_dropout = nn.Dropout(dropout_prob)
        self.gpt_head = nn.Linear(embed_dim*num_heads, num_pred_tokens)
        self.print_n_params()

    @property
    def device(self):
        return self.pos_embed_table.weight.device

    def forward(self, tokens: torch.Tensor) -> torch.Tensor:
        """
        tokens: a (batch, seq_length) long tensor of tokens with values in {0, 1, ..., T-1}, where `T-1` corresponds to <BOS>

        Returns
        - a tensor of logits with shape (batch, seq_length, T-1)
        """
        tok_embd = self.token_embed_table(tokens)    # (batch, seq_length, embed_dim*num_heads)
        pos_embd = self.pos_embed_table.weight       # (seq_length, embed_dim*num_heads)
        x = (torch.cat([tok_embd, pos_embd.unsqueeze(0).expand(tokens.size(0), -1, -1)], dim=2)
             if self.concat_embed
             else tok_embd + pos_embd)               # (batch, seq_length, embed_dim*num_heads)
        x = self.embed_dropout(x)
        for block in self.blocks:
            x = block(x)
        logits = self.gpt_head(x)

        return logits

    def forward_with_cache(self, tokens: torch.Tensor, idx: int):
        """
        tokens: a (batch,) long tensor
        k_cache: a (batch, num_heads, seq_length, embed_dim) tensor
        q_cache: a (batch, num_heads, seq_length, embed_dim) tensor
        idx: an integer corresponding to which part of the cache to fill in

        Returns
        - a tensor of logits with shape (batch, T-1)
        """
        tok_embd = self.token_embed_table(tokens)    # (batch, embed_dim*num_heads)
        pos_embd = self.pos_embed_table.weight[idx]  # (embed_dim*num_heads,)
        x = (torch.cat([tok_embd, pos_embd.unsqueeze(0).expand(tokens.size(0), -1)], dim=1)
             if self.concat_embed
             else tok_embd + pos_embd)               # (batch, embed_dim*num_heads)
        x = self.embed_dropout(x)
        for block in self.blocks:
            x = block.forward_with_cache(x, idx)
        logits = self.gpt_head(x)

        return logits

    def sample_naive(self, n_samples, prompt_token: int, record_times=False):
        """
        Returns
        - a numpy array of shape (n_samples, self.seq_length+1)
        """
        times = []
        with torch.no_grad():
            samples = torch.zeros(n_samples, self.seq_length+1, device=self.device, dtype=torch.long)
            samples[:, 0] = prompt_token
            for i in range(self.seq_length):
                start = time()
                logits = self(samples)[:, i]                                         #  (n_samples, T-1)
                probs = F.softmax(logits, 1)                                         #  (n_samples, T-1)
                sampled_tokens = torch.multinomial(probs, num_samples=1).squeeze(1)  #  (n_samples,)
                end = time()
                samples[:, i+1] = sampled_tokens
                times.append(end-start)
        if record_times:
            return samples.cpu().numpy(), times
        return samples.cpu().numpy()

    def sample(self, n_samples: int, prompt_token: int, record_times: bool = False) -> np.ndarray:
        times = []
        with torch.no_grad():
            samples = torch.zeros(n_samples, self.seq_length+1, device=self.device, dtype=torch.long)
            samples[:, 0] = prompt_token
            for i in range(self.seq_length):
                start = time()
                logits = self.forward_with_cache(samples[:, i], i)                   #  (n_samples, T-1)
                probs = F.softmax(logits, 1)                                         #  (n_samples, T-1)
                sampled_tokens = torch.multinomial(probs, num_samples=1).squeeze(1)  #  (n_samples,)
                end = time()
                samples[:, i+1] = sampled_tokens
                times.append(end-start)
        if record_times:
            return samples.cpu().numpy(), times
        return samples.cpu().numpy()

We now define a tokenizer that converts raw data into a sequence of tokens.

In [ ]:
class Tokenizer:
    def tokenize(self, x: np.ndarray) -> np.ndarray:
        """
        Converts an input to a sequence of tokens.

        x: (batch, *) array

        Returns
        - a (batch, seq_length) long array corresponding to tokens
        """
        raise NotImplementedError()

    def detokenize(self, x: np.ndarray) -> np.ndarray:
        """
        Converts a sequence of tokens to a corresponding output.

        x: (batch, seq_length) long array corresponding to tokens

        Returns
        - a (batch, *) array
        """
        raise NotImplementedError()

In [ ]:
def cartesian_product(arrays):
    # taken from https://stackoverflow.com/a/11146645
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[..., i] = a
    return arr.reshape(-1, la)


class ImageTokenizer(Tokenizer):
    def __init__(self, img_size, num_values):
        """
        img_size: (H, W, C) tuple
        """
        self.H, self.W, self.C = img_size
        self.num_values = num_values
        self.seq_length = self.H*self.W + 1
        self.num_tokens = self.num_values**self.C + 1
        self.detokenize_table = cartesian_product([np.arange(self.num_values) for _ in range(self.C)])

        tokenizer_lst = [1]
        for _ in range(self.C-1):
            tokenizer_lst.append(tokenizer_lst[-1] * self.num_values)
        self.tokenizer_array = np.array(list(reversed(tokenizer_lst)))

    def tokenize(self, x: np.ndarray) -> np.ndarray:
        """
        x: (batch, H, W, C) array
        """
        B, H, W, C = x.shape
        assert H == self.H
        assert W == self.W
        assert C == self.C
        tokens = np.empty([B, self.seq_length], dtype=np.int64)
        tokens[:, 0]  = self.num_tokens-1  # <BOS>
        tokens[:, 1:] = np.einsum('c,bhwc->bhw', self.tokenizer_array, x).reshape(B, self.seq_length-1)
        return tokens

    def detokenize(self, x: np.ndarray) -> np.ndarray:
        """
        Returns
        - a (batch, H, W, C) array
        """
        x = x[:, -(self.seq_length-1):]  # remove any "prompt" elements
        return self.detokenize_table[x].reshape(x.shape[0], self.H, self.W, self.C)

We test the tokenizer on the black and white images.

In [ ]:
train_data, test_data = utils.load_pickled_data("data/shapes.pkl")
print(train_data[:2].squeeze())
print(train_data[:2].shape)

print()
tokenizer = ImageTokenizer(train_data.shape[1:], 2)
tokens = tokenizer.tokenize(train_data[:2])
print(tokens.squeeze())
print(tokens.shape)

print()
x = tokenizer.detokenize(tokens)
print(x.shape)
print((x == train_data[:2]).all())

In [ ]:
make_igpt_model = lambda *args: None

def q3_a(train_data, test_data, image_shape, dset_id):
  """
  train_data: A (n_train, H, W, 1) uint8 numpy array of color images with values in {0, 1}
  test_data: A (n_test, H, W, 1) uint8 numpy array of color images with values in {0, 1}
  image_shape: (H, W, 1), height, width, and # of channels of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (100, H, W, 1) of samples with values in {0, 1}
  """
  pytorch_util.set_gpu_mode("cuda", GPU_ID)
  device = pytorch_util.device
  pytorch_util.seed_rng(0)

  num_values = 2
  tokenizer = ImageTokenizer(image_shape, num_values)
  train_data = tokenizer.tokenize(train_data)
  test_data = tokenizer.tokenize(test_data)
  train_losses, test_losses = [], []

  num_heads, num_layers = 4, 2
  embed_dim = 128*num_heads
  model = make_igpt_model(tokenizer.num_tokens, tokenizer.num_tokens-1, tokenizer.seq_length-1, embed_dim, num_heads, num_layers)
  model.train()
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
  test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

  def evaluate():
    model.eval()
    losses = []
    with torch.no_grad():
      for test_mbatch in test_dataloader:
        test_mbatch = test_mbatch.long().to(device)
        logits = model(test_mbatch[:, :-1])
        tgt = test_mbatch[:, 1:]
        loss = F.cross_entropy(logits.reshape(-1, tokenizer.num_tokens-1), tgt.ravel())
        losses.append(loss.cpu().item())
    model.train()
    return np.mean(losses)

  test_losses.append(evaluate())
  epochs = 15
  train_steps = epochs * len(train_dataloader)
  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=train_steps)
  for _ in tqdm(range(epochs)):
    # train loss
    for train_mbatch in train_dataloader:
      train_mbatch = train_mbatch.long().to(device)
      optimizer.zero_grad()
      logits = model(train_mbatch[:, :-1])
      tgt = train_mbatch[:, 1:]
      loss = F.cross_entropy(logits.reshape(-1, tokenizer.num_tokens-1), tgt.ravel())
      loss.backward()
      optimizer.step()
      scheduler.step()
      train_losses.append(loss.cpu().item())

    # test loss
    test_losses.append(evaluate())

  model.eval()
  samples = tokenizer.detokenize(model.sample(100, tokenizer.num_tokens-1))

  return train_losses, test_losses, samples

### Results

Once you've implemented `q3_a`, execute the cells below to visualize and save your results

In [ ]:
make_igpt_model = lambda *args: GPT(*args, concat_embed=False)

In [ ]:
print("Add embeddings")
q3ab_save_results(1, 'a', q3_a)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Add embeddings")
q3ab_save_results(2, 'a', q3_a)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
make_igpt_model = lambda *args: GPT(*args, concat_embed=True)

In [ ]:
print("Concatenate embeddings")
q3ab_save_results(1, 'a', q3_a)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Concatenate embeddings")
q3ab_save_results(2, 'a', q3_a)
with torch.no_grad():
    torch.cuda.empty_cache()

## Part (b) iGPT on Colored Shapes and MNIST

Now, implement an iGPT that models color. In order to reduce the length of token sequences, iGPT models each RGB pixel as a **single** token. This effectively reduces the context length from $H \cdot W \cdot C$ to just $H \cdot W$. iGPT does this through a k-means clustering approach. Because our images only each can only take on 4 values (2 bits) per channel, we can represent each pixel with 64 values (6 bits). Convert the dataset into an image of tokens and train iGPT on the colored shapes and MNIST dataset.

Checkout the iGPT paper for more details: [Generative Pretraining from Pixels](https://cdn.openai.com/papers/Generative_Pretraining_from_Pixels_V2.pdf) 

Training times and hyperparameter settings should be the same as part (a), except train for longer (15 epochs)

**You will provide these deliverables**

1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. 100 samples from the final trained model


In [ ]:
train_data, test_data = utils.load_pickled_data("data/shapes_colored.pkl")
print(train_data[:2].squeeze())
print(train_data[:2].shape)

print()
tokenizer = ImageTokenizer(train_data.shape[1:], 4)
tokens = tokenizer.tokenize(train_data[:2])
print(tokens.squeeze())
print(tokens.shape)

print()
x = tokenizer.detokenize(tokens)
print(x.shape)
print((x == train_data[:2]).all())

In [ ]:
make_igpt_model = lambda *args: None
save_model = False

def q3_b(train_data, test_data, image_shape, dset_id):
  """
  train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  image_shape: (H, W, C), height, width, and # of channels of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (100, H, W, C) of samples with values in {0, 1, 2, 3}
  """
  pytorch_util.set_gpu_mode("cuda", GPU_ID)
  device = pytorch_util.device
  pytorch_util.seed_rng(0)

  num_values = 4
  tokenizer = ImageTokenizer(image_shape, num_values)
  train_data = tokenizer.tokenize(train_data)
  test_data = tokenizer.tokenize(test_data)
  train_losses, test_losses = [], []

  num_heads, num_layers = 4, 2
  embed_dim = 128*num_heads
  model = make_igpt_model(tokenizer.num_tokens, tokenizer.num_tokens-1, tokenizer.seq_length-1, embed_dim, num_heads, num_layers)
  model.train()
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
  test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

  def evaluate():
    model.eval()
    losses = []
    with torch.no_grad():
      for test_mbatch in test_dataloader:
        test_mbatch = test_mbatch.long().to(device)
        logits = model(test_mbatch[:, :-1])
        tgt = test_mbatch[:, 1:]
        loss = F.cross_entropy(logits.reshape(-1, tokenizer.num_tokens-1), tgt.ravel())
        losses.append(loss.cpu().item())
    model.train()
    return np.mean(losses)
  
  test_losses.append(evaluate())
  epochs = 15
  train_steps = epochs * len(train_dataloader)
  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=train_steps)
  for _ in tqdm(range(epochs)):
    # train loss
    for train_mbatch in train_dataloader:
      train_mbatch = train_mbatch.long().to(device)
      optimizer.zero_grad()
      logits = model(train_mbatch[:, :-1)
      tgt = train_mbatch[:, 1:]
      loss = F.cross_entropy(logits.reshape(-1, tokenizer.num_tokens-1), tgt.ravel())
      loss.backward()
      optimizer.step()
      scheduler.step()
      train_losses.append(loss.cpu().item())

    # test loss
    test_losses.append(evaluate())

  if save_model:
    torch.save(model.state_dict(), "data/igpt_model.pth")

  model.eval()
  samples = tokenizer.detokenize(model.sample(100, tokenizer.num_tokens-1))
    
  return train_losses, test_losses, samples

### Results

Once you've implemented `q3_b`, execute the cells below to visualize and save your results

In [ ]:
make_igpt_model = lambda *args: GPT(*args, concat_embed=False)

In [ ]:
print("Add embeddings")
save_model = False
q3ab_save_results(1, 'b', q3_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Add embeddings")
save_model = True
q3ab_save_results(2, 'b', q3_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
make_igpt_model = lambda *args: GPT(*args, concat_embed=True)

In [ ]:
print("Concatenate embeddings")
save_model = False
q3ab_save_results(1, 'b', q3_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
print("Concatenate embeddings")
save_model = False
q3ab_save_results(2, 'b', q3_b)
with torch.no_grad():
    torch.cuda.empty_cache()

## Part (c) K, V Caching for Improved Inference
You may have noticed that generation from the transformer is quite slow. Part of this is just due to the autoregressive nature. However, another part is due to some computational inefficiency. At each forward pass of the model, we are performing repeat computation of the past sequence. Specifically, we can cache the key and values at the multi attention layer to more quickly predict at each step.

In self-attention, a sequence is processed by generating three vectors for each element in the sequence: a Query (Q), a Key (K), and a Value (V). These vectors are then used to compute attention scores and subsequently the output of the attention layer.
Mathematically, this can be represented as:
 * For each index $i$, compute $Q_i$, $K_i$, $V_i$ for the current element
 * Retrieve $K_{<i}$ and $V_{<i}$ from the cache (where $<i$ denotes all indices before the current one)
 * Compute the attention output using $Q_i$, $[K_{<i}, K_i]$, $[V_{<i}, V_i]$


Next implement caching for your transformer to make inference more efficient by modifying your self attention. Use caching for inference in the future problems for faster generation! (Note caching is only used during inference). You will use the same dataset as in part B, dataset 2 of this question (colored mnist). No training is required in this section, feel free to reuse the model you trained in part B, dataset 2.

**You will provide these deliverables**

1. Over the course of inference, measure the time for the forward pass over the total sequence length with and without caching.
3. 100 samples from the final trained model using the caching inference pipeline.



In [ ]:
def q3_c(train_data, test_data, image_shape, dset_id):
    """
    train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
    test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
    image_shape: (H, W, C), height, width, and # of channels of the image
    dset_id: An identifying number of which dataset is given (1 or 2). Most likely
             used to set different hyperparameters for different datasets

    Returns
    - a (# sampling steps,) numpy array of time per sampling iteration, without caching
    - a (# sampling steps,) numpy array of time per sampling iteration, with caching
    - a numpy array of size (100, H, C, W) of samples with values in {0, 1, 2, 3} (sample generated without caching)
    - a numpy array of size (100, H, C, W) of samples with values in {0, 1, 2, 3} (sample generated with caching)
    """
    pytorch_util.set_gpu_mode("cuda", GPU_ID)
    device = pytorch_util.device

    # tokenizer
    num_values = 4
    tokenizer = ImageTokenizer(image_shape, num_values)

    # model
    num_heads, num_layers = 4, 2
    embed_dim = 128*num_heads
    model = GPT(tokenizer.num_tokens, tokenizer.num_tokens-1, tokenizer.seq_length-1, embed_dim, num_heads, num_layers)
    model.load_state_dict(torch.load("data/igpt_model.pth"))
    model.eval()
    model.to(device)

    # naive sampling
    pytorch_util.seed_rng(0)
    samples, time_list_no_cache = model.sample_naive(100, tokenizer.num_tokens-1, True)
    samples_no_cache = tokenizer.detokenize(samples)

    # sampling with KV cache
    pytorch_util.seed_rng(0)
    samples, time_list_with_cache = model.sample(100, tokenizer.num_tokens-1, True)
    samples_with_cache = tokenizer.detokenize(samples)

    return time_list_no_cache, time_list_with_cache, samples_no_cache, samples_with_cache

### Results

Once you've implemented `q3_c`, execute the cells below to visualize and save your results



In [ ]:
q3c_save_results(2, q3_c)
with torch.no_grad():
    torch.cuda.empty_cache()

# Question 4: Causal Transformer: Tokenized Images

## Image Tokenization with Vector Quanization

## Part (a) Image Quantization

Above, we implemented iGPT, which autoregressivly predicts raw pixels. Transformers have quadratic complexity in the sequence length which prevents this naive approach from scaling well to large images.

The space of natural images often contains very correlated information. This suggests we can learn a reduced representation. VQVAE is a method that does just that, learning to map images to a more compact discrete set of tokens. We will cover this method in more detail in future lectures. The only thing you need to know now is that we can learn an encoder (and corresponding decoder), which can extract a discrete representation from an image. 

If you are curious, checkout the VQVAE paper to learn more: https://arxiv.org/abs/1711.00937 (we will cover this in a future lecture though!)

In this part, we provide a pre-trained VQVAE model, which consists of:
 * encoder to tokenize the images
 * the decoder to recover the image
 * a token vocabulary of VQVAE_MODEL.n_embeddings

Below is the code for loading the VQ model. Note that VQVAE encoding process is lossy, so the decoded images will not be the exact same as the input. Some blurriness in the recovered image is to be expected. The docstrings of the relevant methods you will need for the VQVAE_MODEL are provided below for your convenience. 

We will use 2 colored mnist datasets in this part. The first is the same dataset used in previous parts. The second, hads a colored digit on a differently colored background. We will call these datasets Colored MNIST and Colored MNIST v2. Note that the vqvae is trained per dataset.

**You will provide these deliverables**

1. Use the provided encoder model to quantize the images then inspect the recovered images by applying the decoder for each of the two datasets

In [ ]:
# @property
# def n_embeddings(self) -> int:
#     """The size of the token vocabulary"""
#    
# def quantize(self, x: np.ndarray) -> np.ndarray:
#     """Quantize an image x.
#
#     Args:
#         x (np.ndarray, dtype=int): Image to quantize. shape=(batch_size, 28, 28, 3). Values in [0, 3].
#
#     Returns:
#         np.ndarray: Quantized image. shape=(batch_size, 7, 7). Values in [0, n_embeddings]
#     """
#    
# def decode(self, z_index: np.ndarray) -> np.ndarray:
#     """Decode a quantized image.
#
#     Args:
#         z_index (np.ndarray, dtype=int): Quantized image. shape=(batch_size, 7, 7). Values in [0, n_embeddings].
#
#     Returns:
#         np.ndarray: Decoded image. shape=(batch_size, 28, 28, 3). Values in [0, 3].
#     """
# 

In [ ]:
def q4_a(images, vqvae):
  """
  images: (B, H, W, C), the images to pass through the encoder and decoder of the vqvae
  vqvae: a vqvae model, trained on the relevant dataset

  Returns
  - a numpy array of size (B, H, W, C) of the decoded image
  """
  print("Token vocabulary size:", vqvae.n_embeddings)
  encoded_images = vqvae.quantize(images)
  decoded_images = vqvae.decode(encoded_images)
  return decoded_images

In [ ]:
q4a_save_results(1, q4_a)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
q4a_save_results(2, q4_a)
with torch.no_grad():
    torch.cuda.empty_cache()

## Part (b) Autoregressive Transformer on Colored Shapes and MNIST with Vector Quantization

We can use the VQVAE to tokenize an image dataset. This will result in a much smaller sequence length than the approach we tried in Question 3(b). For this part, train a transformer on the dataset tokenized by the VQVAE.

This is a simplified version of the approach used in VQGAN [VQGAN](https://arxiv.org/abs/2012.09841) -> Section 3.2: Learning the Composition of Images with Transformers (Again, we will cover this in more detail in a future lecture!)

Update the following hyperparameters:
* layers: 4 (we can train a bigger transformer now since less memory is used per input!)
* 30 epochs

**You will provide these deliverables**

1. Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2. Report the final test set performance of your final model
3. 100 samples from the final trained model

In [ ]:
def q4_b(train_data, test_data, image_shape, dset_id, vqvae):
  """
  train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  image_shape: (H, W, C), height, width, and # of channels of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets
  vqvae: a vqvae model, trained on dataset dset_id

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (100, H, C, W) of samples with values in {0, 1, 2, 3}
  """
  pytorch_util.set_gpu_mode("cuda", GPU_ID)
  device = pytorch_util.device
  pytorch_util.seed_rng(0)

  encoded_train_data = np.expand_dims(vqvae.quantize(train_data), axis=3)
  encoded_test_data  = np.expand_dims(vqvae.quantize(test_data),  axis=3)

  tokenizer = ImageTokenizer(encoded_train_data.shape[1:], vqvae.n_embeddings)
  tok_train_data = tokenizer.tokenize(encoded_train_data)
  tok_test_data = tokenizer.tokenize(encoded_test_data)
  train_losses, test_losses = [], []

  num_heads, num_layers = 4, 4
  embed_dim = 32*num_heads
  dropout_prob = 0.1
  model = GPT(tokenizer.num_tokens, tokenizer.num_tokens-1, tokenizer.seq_length-1, embed_dim, num_heads, num_layers, dropout_prob=dropout_prob)
  model.train()
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  train_dataloader = DataLoader(tok_train_data, batch_size=256, shuffle=True)
  test_dataloader = DataLoader(tok_test_data, batch_size=256, shuffle=False)

  def evaluate():
    model.eval()
    losses = []
    with torch.no_grad():
      for test_mbatch in test_dataloader:
        test_mbatch = test_mbatch.long().to(device)
        logits = model(test_mbatch[:, :-1])
        tgt = test_mbatch[:, 1:]
        loss = F.cross_entropy(logits.reshape(-1, tokenizer.num_tokens-1), tgt.ravel())
        losses.append(loss.cpu().item())
    model.train()
    return np.mean(losses)
  
  test_losses.append(evaluate())
  epochs = 30
  train_steps = epochs * len(train_dataloader)
  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=train_steps)
  for i in tqdm(range(epochs)):
    # train loss
    for train_mbatch in train_dataloader:
      train_mbatch = train_mbatch.long().to(device)
      optimizer.zero_grad()
      logits = model(train_mbatch[:, :-1])
      tgt = train_mbatch[:, 1:]
      loss = F.cross_entropy(logits.reshape(-1, tokenizer.num_tokens-1), tgt.ravel())
      loss.backward()
      optimizer.step()
      scheduler.step()
      train_losses.append(loss.cpu().item())

    # test loss
    test_losses.append(evaluate())

  model.eval()
  samples = vqvae.decode(tokenizer.detokenize(model.sample(100, tokenizer.num_tokens-1)).squeeze(3))
    
  return train_losses, test_losses, samples

### Results

Once you've implemented `q4_b`, execute the cells below to visualize and save your results



In [ ]:
q4b_save_results(1, q4_b)
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
q4b_save_results(2, q4_b)
with torch.no_grad():
    torch.cuda.empty_cache()

# Question 5: Causal Transformer: Text

Now lets consider text! You are probably already fimilar with autoregressive transformers for text, now more commonly known as Large Language Modesl (LLMs).
We will now implement a simplified version.

We will be detailing with a [small poetry dataset](https://huggingface.co/datasets/merve/poetry). See some of the data below.

In [ ]:
data = visualize_q5_data()

## Part (a) Modeling Text
Train a transformer on the poetry dataset.

Data Preprocessing:
* We will use a simple method to tokenize the data. We will convert each unique character into a token. (Current LLMs use more sophisticated tokenizers, most commonly, [byte-pair encoding](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt))
* Previously we have leveraged a **\<bos\>** as part of the model, just like iGPT. For text, we may not always sample a sequence that starts at the beginning. Instead, we will add the **\<bos\>** token to the beginning of every sequence in the dataset, and remove the **\<bos\>** token from the model.
* Another problem is that the model must know when to stop sampling. This is done by appending an **\<eos\>**, or end of sequence token at the end of every sequence in the dataset.
* We can now convert the sequence into subsequences of size context_length, for training!

We recommend the following hyperparameters:
* Sequence length: 128
* 5 epochs

**You will provide these deliverables**

1. Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2. Report the final test set performance of your final model
3. Provide **5 unconditional samples** of **128 characters** showcasing the model text generation capabilities (text samples should stop after **\<eos\>**. Text after **\<eos\>** can be removed in post processing)

In [ ]:
dir_path = utils.get_data_dir(1)
train_text, _ = load_text_data(osp.join(dir_path, "poetry.pkl"))

In [ ]:
print("Number of text snippets:", len(train_text))
print("Number of letters in first snippet:", len(train_text[0]))
chars = sorted(list(set("".join(train_text))))
print("Number of unique characters:", len(chars))
print("Unique characters:", chars)

In [ ]:
class TextTokenizer(Tokenizer):
    def __init__(self, chars):
        self.num_tokens = len(chars)+3
        self.bos = "<BOS>"
        self.eos = "<EOS>"
        self.eos_tok = self.num_tokens-3
        self.bos_tok = self.num_tokens-2
        self.ignore_tok = self.num_tokens-1

        self.stoi = {s: i for i, s in enumerate(chars)}
        self.stoi[self.eos] = self.eos_tok
        self.stoi[self.bos] = self.bos_tok
        self.itos = {i: s for s, i in self.stoi.items()}

    def tokenize(self, list_of_strings):
        list_of_tokens = []
        for string in list_of_strings:
            tokens = [self.bos_tok] + [self.stoi[s] for s in string] + [self.eos_tok]
            list_of_tokens.append(tokens)
        return list_of_tokens

    def detokenize(self, array_of_tokens):
        list_of_strings = []
        for tokens in array_of_tokens:
            chars = []
            for token in tokens:
                if token == self.bos_tok:
                    continue
                if token == self.eos_tok:
                    break
                chars.append(self.itos[token])
            list_of_strings.append(''.join(chars))
        return list_of_strings

In [ ]:
tokenizer = TextTokenizer(chars)

In [ ]:
train_tokens = tokenizer.tokenize(train_text)

In [ ]:
print("Text sample:", train_text[0][:10])
print("Tokenization of text sample:", train_tokens[0][:11])

In [ ]:
class TokenDataset(data.Dataset):
    """
    Represents a dataset of heterogenous-length token sequences.
    """
    def __init__(self, list_of_tokens, seq_length):
        self.seq_length = seq_length
        self.list_of_tokens = [self._pad_and_tensorize(tokens) for tokens in list_of_tokens]
        self.lookup_table = self._make_lookup_table()

    def __len__(self):
        return self.lookup_table.shape[0]

    def __getitem__(self, idx):
        string_idx, start, end = self.lookup_table[idx]
        return self.list_of_tokens[string_idx][start:end]

    def _pad_and_tensorize(self, tokens):
        if len(tokens) < self.seq_length:
            tokens = tokens + (self.seq_length-len(tokens)) * [tokens[-1]+2]
        return torch.tensor(tokens)

    def _make_lookup_table(self):
        """
        idx -> (string_idx, start, end)
        """
        arrays = []
        for string_idx, tokens in enumerate(self.list_of_tokens):
            starts = np.arange(len(tokens)-self.seq_length+1)
            ends = starts + self.seq_length
            arrays.append(np.stack([np.full(len(starts), string_idx), starts, ends], axis=1))
        return np.concatenate(arrays)

In [ ]:
dataset = TokenDataset(train_tokens, 128)

In [ ]:
snippet_lens = np.array(list(map(len, train_text)))
print("Shortest text snippet length:", snippet_lens.min())
text_idx = snippet_lens.argmin()

In [ ]:
idx = (dataset.lookup_table[:,0] == text_idx).argmax()
print("Corresponding tokenization:")
print(dataset[idx])

In [ ]:
def q5_a(train_text, test_text):
    """
    train_text: list[str] Train text sequences.
    test_text: list[str] Test text sequences.

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a list of 5 (str), 5 generated samples from the model.
    """
    pytorch_util.set_gpu_mode("cuda", GPU_ID)
    device = pytorch_util.device
    pytorch_util.seed_rng(0)

    chars = sorted(list(set("".join(train_text))))
    tokenizer = TextTokenizer(chars)
    train_tokens = tokenizer.tokenize(train_text)
    test_tokens = tokenizer.tokenize(test_text)
    train_losses, test_losses = [], []

    num_heads, num_layers = 4, 2
    embed_dim = 16*num_heads # 32*num_heads
    seq_length = 128
    dropout_prob = 0.1
    model = GPT(tokenizer.num_tokens, tokenizer.num_tokens-2, seq_length-1, embed_dim, num_heads, num_layers, dropout_prob=dropout_prob)
    model.train()
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    train_dataset = TokenDataset(train_tokens, seq_length)
    test_dataset = TokenDataset(test_tokens, seq_length)
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    def evaluate():
        model.eval()
        losses = []
        with torch.no_grad():
            for test_mbatch in test_dataloader:
                test_mbatch = test_mbatch.long().to(device)
                logits = model(test_mbatch[:, :-1])
                tgt = test_mbatch[:, 1:]
                loss = F.cross_entropy(logits.reshape(-1, tokenizer.num_tokens-2), tgt.ravel(), ignore_index=tokenizer.ignore_tok)
                losses.append(loss.cpu().item())
        model.train()
        return np.mean(losses)

    test_losses.append(evaluate())
    epochs = 20 # 10
    train_steps = epochs * len(train_dataloader)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=train_steps)
    for _ in tqdm(range(epochs)):
        # train loss
        for train_mbatch in train_dataloader:
            train_mbatch = train_mbatch.long().to(device)
            optimizer.zero_grad()
            logits = model(train_mbatch[:, :-1])
            tgt = train_mbatch[:, 1:]
            loss = F.cross_entropy(logits.reshape(-1, tokenizer.num_tokens-2), tgt.ravel(), ignore_index=tokenizer.ignore_tok)
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_losses.append(loss.cpu().item())

        # test loss
        test_losses.append(evaluate())

    torch.save(model.state_dict(), "data/text_gpt.pth")
    model.eval()
    token_samples = model.sample(5, tokenizer.num_tokens-2)
    text_samples = tokenizer.detokenize(token_samples)
            
    return train_losses, test_losses, text_samples

### Results

Once you've implemented `q5_a`, execute the cells below to visualize and save your results



In [ ]:
q5a_save_results(q5_a)
with torch.no_grad():
    torch.cuda.empty_cache()

# Question 6: Causal Transformer: Multimodal

So far, we have been dealing only with autoregressive generation of a single modality. Now we will train a model that operates on multiple modalities!

We will use the text labeled colored MNIST dataset, which has a text description of the MNIST image. Run the cell below to visualize the data along with the text annotation. This is the Colored MNIST v2 dataset, which also comes with these text labels.

In [ ]:
visualize_q6_data()

## Part (a) Multimodal Text and Image Generation
Implement and train an autoregressive (AR) model capable of handling both text and image data. The model should be designed to process sequences composed of concatenated text and image tokens in both orders (text followed by images and images followed by text). Additionally, the model should be capable of generating unconditional text and image samples.

Data Preprocessing:
* Text Tokens: Map each unique word in the text data to a unique token. (Note that all text descriptions contain the exact same amount of words. This simplifies text processing, as you won't have to deal with sequences of different lengths as in Question 5)
* Image Tokens: Quantize the image data into tokens using the VQVAE tokenizer from Problem 4.
* In this problem, we have 2 modalities. Introduce an **\<end of text\>** token and an **\<end of image\>** token. After seeing such a token, the model should switch to sampling the next modality.
* Formulate batches as sequences of concat([**\<end of image\>**, text_tokens, **\<end of text\>**, image_tokens]) and concat([**\<end of text\>**, image_tokens, **\<end of image\>**, text_tokens]). With a 50/50 split between each ordering.

Inference:
* During inference, we cannot mix modality tokens. During sampling we can restrict the logits to only be within the relevant modality.
* After **\<end of image\>**, only allow the model to sample text tokens (including **\<end of text\>**)
* After **\<end of text\>**, only allow the model to sample image tokens (including **\<end of image\>**)
* At the very start (conditioned on the **\<bos\>** token, only allow the model to sample one of (**\<end of image\>** or **\<end of text\>**))
* As the model may not always correctly sample the **\<end of image\>** token before the image ends, you may add a rule to force the model to always sample the correct number of image tokens (49 tokens).

You can use the same hyperparameters as in 4(b) (but of course, feel free to tune your model to achieve better performance)

**You will provide these deliverables**

1. Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2. Report the final test set performance of your final model
3. 9 conditional samples based on provided text.
4. 9 conditional samples based on provided images.
5. 9 unconditional samples showcasing the model's capability in generating standalone text and images.

In [ ]:
def q6_a(train_data, test_data, image_shape, train_text, test_text, image_test_prompt, text_test_prompt, vqvae):
    """
    train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
    test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
    image_shape: tuple (H, W, C) The shape of the images in the dataset, indicating height, width, and number of color channels.
    train_text: list[str] Text data associated with each training image.
    test_text: list[str] Text data associated with each test image.
    image_test_prompt: (9, H, W, C) Image data used for generating conditional text samples during testing.
    text_test_prompt: list of 9 strings Text prompts used for generating conditional image samples during testing.
    vqvae: a vqvae model, trained on the relevant dataset

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a list of 9 (image, text), corresponding to the image conditioned samples
    - a list of 9 (image, text), corresponding to the text conditions samples
    - a list of 9 (image, text), corresponding to unconditional samples
    """
    return train_losses, test_losses, samples_image_conditioned, samples_text_conditioned, samples_unconditioned

### Results

Once you've implemented `q6_a`, execute the cells below to visualize and save your results



In [ ]:
q6a_save_results(q6_a)
with torch.no_grad():
    torch.cuda.empty_cache()